# Tutorial

An example of using Theatre_Ag to model the establishment of stateful TCP connections over an IP network.

In [ ]:
!pip install git+https://github.com/twsswt/theatre_ag

First, we define a Directions class to allocate tasks to the actors performing as the TCP client and server:

In [ ]:
from theatre_tcp_ip import TCPClient, TCPServer
from theatre_tcp_ip import Network

class TCPDirections(object):
    """
    The setup script for running a simulation of TCP behaviour within this test.
    """

    def __init__(self, network, tcp_server_address):
        self.network = network
        self.tcp_server_address = tcp_server_address

    def apply(self, cast):

        tcp_server = filter(lambda m: m.logical_name is self.tcp_server_address, cast)[0]
        server_nix = self.network.create_network_endpoint(self.tcp_server_address)
        tcp_server_workflow = TCPServer(server_nix)
        tcp_server.allocate_task(tcp_server_workflow.wait_for_syns)

        tcp_clients = filter(lambda m: m.logical_name is not self.tcp_server_address, cast)

        for tcp_client in tcp_clients:
            client_nix = self.network.create_network_endpoint(tcp_client.logical_name)
            tcp_client_workflow = TCPClient(client_nix, self.tcp_server_address)
            tcp_client.allocate_task(tcp_client_workflow.send_syn)


Next, we define the simulation clock with a maximum tick:

In [ ]:
from theatre_ag import SynchronizingClock
clock = SynchronizingClock(3)

Define the cast:

In [ ]:
from theatre_ag import TaskQueueActor, Cast

tcp_client = TaskQueueActor('tcp_client', clock)
tcp_server = TaskQueueActor('tcp_server', clock)
cast = Cast({tcp_client, tcp_server})

Define the scene the cast will operate on (the IP network)

In [ ]:
network = Network()

Finally we use the directions and an episode to link the scene and cast together and perform the simulation:

In [ ]:
from theatre_ag import Episode

directions = TCPDirections(network, 'tcp_server')

episode = Episode(clock, cast, directions)

episode.perform()

It is possible to query the state of the simulation, include the task history of the actors:

In [ ]:
from theatre_ag import format_task_trees

print format_task_trees(tcp_client.task_history)
print format_task_trees(tcp_server.task_history)